In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Константы и параметры
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
BATCH_SIZE = 16
MAX_LENGTH = 256
NUM_EPOCHS = 15
LEARNING_RATE = 2e-5
PATIENCE = 4

# Загрузка данных
try:
    data = pd.read_csv('processed_data.csv')
    logging.info("Данные успешно загружены.")
except Exception as e:
    logging.error(f"Ошибка при загрузке данных: {e}")
    raise

categories = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]

# Проверка наличия категорий в данных и подготовка меток
try:
    labels = data[categories].values.astype(int)
except KeyError as e:
    logging.error(f"Некоторые категории не найдены в данных: {e}")
    raise

# Разделение данных с стратификацией по первому классу для сбалансированности выборки
try:
    stratify_labels = labels.argmax(axis=1)
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        data['comment'].values,
        labels,
        test_size=0.2,
        random_state=42,
        stratify=stratify_labels
    )
except Exception as e:
    logging.error(f"Ошибка при разделении данных: {e}")
    raise

# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

class CommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length= max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text= str(self.texts[idx])
        label= self.labels[idx]
        encoding= self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.FloatTensor(label)
        }

# Создание датасетов и загрузчиков данных
train_dataset = CommentsDataset(train_texts, train_labels, tokenizer)
val_dataset = CommentsDataset(val_texts, val_labels, tokenizer)

# Расчет весов классов для балансировки потерь и выборки (чтобы бороться с дисбалансом)
class_counts = train_labels.sum(axis=0)
total_samples = len(train_labels)
epsilon=1e-6  # чтобы избежать деления на ноль

# Используем логарифм для взвешивания (можно оставить или изменить по необходимости)
class_weights_np= np.log((total_samples / (class_counts + epsilon)))
class_weights_tensor= torch.FloatTensor(class_weights_np).to('cpu')  # для вычислений

# Веса для выборки (WeightedRandomSampler) — чтобы сбалансировать обучение по классам
sample_weights=[]
for label in train_labels:
    class_indices= np.where(label==1)[0]
    if len(class_indices)>0:
        weights_for_classes= class_weights_np[class_indices]
        sample_weight= np.min(weights_for_classes)  # можно выбрать среднее или максимум по необходимости
    else:
        sample_weight=1.0  # если класс отсутствует в примере
    
    sample_weights.append(sample_weight)

sample_weights=np.array(sample_weights)
sampler= WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader= DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
val_loader= DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Определение модели с несколькими выходами (multi-label classification)
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(categories))
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)  
        return logits

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertMultiLabelClassifier().to(device)

import torch.nn.functional as F

# Исправление: FocalLoss не принимает pos_weight как аргумент конструктора.
# Вместо этого используем pos_weight внутри forward.
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super().__init__()
        self.alpha=alpha
        self.gamma=gamma
        self.reduction=reduction
    
    def forward(self, inputs, targets):
        # inputs: logits (не после sigmoid)
        # targets: бинарные метки (0 или 1)
        
        bce_loss=F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        
        probs=torch.sigmoid(inputs)
        
        # Для каждого элемента выбираем prob или 1-prob в зависимости от targets.
        pt=torch.where(targets==1., probs, 1.-probs)
        
        focal_weight=(self.alpha)*(1.-pt)**self.gamma
        
        loss=focal_weight * bce_loss
        
        if self.reduction=='mean':
            return loss.mean()
        elif self.reduction=='sum':
            return loss.sum()
        else:
            return loss

# Создаем экземпляр потерь без pos_weight (его можно встроить внутри forward при необходимости).
criterion=FocalLoss()

optimizer=optim.AdamW(model.parameters(), lr=LEARNING_RATE)

total_steps=len(train_loader)*NUM_EPOCHS

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1*total_steps),
    num_training_steps=total_steps
)

best_f1_micro=0
epochs_without_improvement=0

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss=0
    
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", unit='batch') as pbar:
        for batch in train_loader:
            input_ids=batch['input_ids'].to(device)
            attention_mask=batch['attention_mask'].to(device)
            labels=batch['labels'].to(device)

            optimizer.zero_grad()
            outputs=model(input_ids=input_ids, attention_mask=attention_mask)
            loss=criterion(outputs, labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # градиентный клиппинг
            
            optimizer.step()
            scheduler.step()

            total_loss+=loss.item()
            pbar.update(1)

    print(f"\nЭпоха {epoch+1} завершена. Средний Loss: {total_loss/len(train_loader):.4f}")

    # Валидация и метрики после каждой эпохи
    model.eval()
    
    all_preds=[]
    all_true=[]
    
    for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}", unit='batch'):
        with torch.no_grad():
            input_ids=batch['input_ids'].to(device)
            attention_mask=batch['attention_mask'].to(device)

            outputs=model(input_ids=input_ids, attention_mask=attention_mask)
            preds=torch.sigmoid(outputs).cpu().numpy()

            all_preds.extend(preds)
            all_true.extend(batch['labels'].cpu().numpy())

    true_labels=np.array(all_true) 
    preds_array=np.array(all_preds) 

    pred_labels=(preds_array>=0.5).astype(int)

    # В конце каждой эпохи после вычисления F1
    try:
        f1_micro = f1_score(true_labels, pred_labels, average='micro')
        
        # Сохраняем модель при улучшении F1 micro.
        if f1_micro > best_f1_micro:
            best_f1_micro = f1_micro
            torch.save(model.state_dict(), 'best_f1micro.pth')
        
        print(f"F1-макро на эпохе {epoch+1}: {f1_micro:.4f}")

        for i, cate in enumerate(categories):
            score = f1_score(true_labels[:, i], pred_labels[:, i])
            print(f"F1-score для '{cate}': {score:.4f}")

        # Ранняя остановка при отсутствии улучшений по F1 micro.
        if f1_micro > best_f1_micro:
            epochs_without_improvement = 0  # сбрасываем счетчик при улучшении.
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= PATIENCE:
            print("Достигнут patience без улучшения — остановка обучения.")
            break

    except Exception as e:
        logging.error(f"Ошибка во время оценки или обучения: {e}")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 254)

In [ ]:
# Оценка модели на валидационной выборке с обработкой ошибок и логированием

try:
    model.load_state_dict(torch.load('best_f1micro.pth'))
    logging.info("Модель загружена.")
except Exception as e:
    logging.warning(f"Не удалось загрузить модель: {e}")

model.eval()

all_preds = []
all_true = []

try:
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].cpu().numpy()

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.sigmoid(outputs).cpu().numpy()

                all_preds.extend(preds)
                all_true.extend(labels)

            except Exception as e:
                logging.warning(f"Ошибка при обработке батча {batch_idx}: {e}")
except Exception as e:
    logging.error(f"Ошибка во время оценки: {e}")

true_labels = np.array(all_true)  # shape: (num_samples,num_classes)
preds_array = np.array(all_preds)

pred_labels = (preds_array >= 0.5).astype(int)

try:
    f1_micro = f1_score(true_labels, pred_labels, average='micro')
except Exception as e:
    logging.error(f"Ошибка при вычислении F1-score: {e}")

# Расчет по классам
f1_class_scores = []
for i, cate in enumerate(categories):
    try:
        score = f1_score(true_labels[:, i], pred_labels[:, i])
        f1_class_scores.append(score)
        print(f"F1-score для '{cate}': {score:.4f}")
    except Exception as e:
        logging.warning(f"Ошибка при вычислении F1 для '{cate}': {e}")

# Вывод средних значений F1
try:
    print(f"F1-score (micro): {f1_micro:.4f}")
except NameError:
    print("Некорректные значения F1-score.")

print('--------------------------------------------------------------------------')

# ROC-AUC по классам с обработкой ошибок
roc_auc_scores = []
for i, cate in enumerate(categories):
    try:
        score = roc_auc_score(true_labels[:, i], preds_array[:, i])
        roc_auc_scores.append(score)
        print(f"ROC-AUC для '{cate}': {score:.4f}")
    except ValueError as e:
        roc_auc_scores.append(None)
        print(f"ROC-AUC для '{cate}': недоступен ({e})")
        
if any(score is not None for score in roc_auc_scores):
    valid_scores = [score for score in roc_auc_scores if score is not None]
    roc_auc_mean = np.mean(valid_scores) if valid_scores else None
else:
    roc_auc_mean = None

if roc_auc_mean is not None:
    print(f"\nСредний ROC-AUC по классам: {roc_auc_mean:.4f}")
else:
     print("\nНет доступных значений ROC-AUC для вычисления среднего.")

F1-score для 'Вопрос решен': 0.4950
F1-score для 'Нравится качество выполнения заявки': 0.5294
F1-score для 'Нравится качество работы сотрудников': 0.8931
F1-score для 'Нравится скорость отработки заявок': 0.8987
F1-score для 'Понравилось выполнение заявки': 0.4301
F1-score для 'Другое': 0.7805

Средний F1-score (macro): 0.6711
F1-score (micro): 0.7367
--------------------------------------------------------------------------
ROC-AUC для 'Вопрос решен': 0.7590
ROC-AUC для 'Нравится качество выполнения заявки': 0.8845
ROC-AUC для 'Нравится качество работы сотрудников': 0.9663
ROC-AUC для 'Нравится скорость отработки заявок': 0.9593
ROC-AUC для 'Понравилось выполнение заявки': 0.7998
ROC-AUC для 'Другое': 0.9196

Средний ROC-AUC по классам: 0.8814
